In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt

# 파일 불러오기
or_data = pd.read_csv('./셀바스AI 주식 데이터_preprocess.csv', encoding='euc-kr')

In [31]:
print(or_data.head())
print(or_data.shape)
print(or_data.info())
print(or_data.columns)
print(or_data.index)
print(or_data.values)
print(or_data.describe().T)
or_data.isnull().sum()

      종목코드    종목명        날짜   시간      시가      고가      저가      종가     거래량  \
0  A108860  셀바스AI  20210709  901  4421.0  4435.0  4411.0  4435.0   973.0   
1  A108860  셀바스AI  20210709  902  4411.0  4411.0  4372.0  4372.0  2718.0   
2  A108860  셀바스AI  20210709  903  4372.0  4377.0  4358.0  4358.0  3614.0   
3  A108860  셀바스AI  20210709  904  4358.0  4358.0  4329.0  4329.0  5120.0   
4  A108860  셀바스AI  20210709  905  4324.0  4324.0  4276.0  4314.0  7357.0   

     SMA5   SMA20        UPPER     MAVG        LOWER       RSI       MACD  \
0  4361.6  4366.6  4420.211934  4363.75  4307.288066  34.87395 -15.880418   
1  4361.6  4366.6  4420.211934  4363.75  4307.288066  34.87395 -15.880418   
2  4361.6  4366.6  4420.211934  4363.75  4307.288066  34.87395 -15.880418   
3  4361.6  4366.6  4420.211934  4363.75  4307.288066  34.87395 -15.880418   
4  4361.6  4366.6  4420.211934  4363.75  4307.288066  34.87395 -15.880418   

     AROONUP  AROONDN  
0  28.571429      0.0  
1  28.571429      0.0  
2  28.57

종목코드       0
종목명        0
날짜         0
시간         0
시가         0
고가         0
저가         0
종가         0
거래량        0
SMA5       0
SMA20      0
UPPER      0
MAVG       0
LOWER      0
RSI        0
MACD       0
AROONUP    0
AROONDN    0
dtype: int64

In [ ]:
# COLUMNS_CHART_DATA = ['날짜', '시간', '시가', '고가', '저가', '종가', '거래량']

# COLUMNS_TRAINING_DATA_V1 = [
#     'SMA5',	
#     'SMA20',	
#     'UPPER',	
#     'MAVG',	
#     'LOWER',	
#     'RSI',	
#     'MACD',	
#     'AROONUP',	
#     'AROONDN'
#     ]

In [33]:
def generate_custom_code(charts_data, training_data):
    # 데이터 전처리 - 차트/트레이닝 데이터 구분
    charts_data = pd.DataFrame(or_data, columns=['날짜', '시간', '시가', '고가', '저가', '종가', '거래량'])
    training_data = pd.DataFrame(or_data, columns=['SMA5', 'SMA20', 'UPPER', 'MAVG', 'LOWER', 'RSI', 'MACD', 'AROONUP', 'AROONDN'])
    data = pd.concat([charts_data, training_data], axis=1) # 다시 결합
    
    # 입력 데이터, 출력 데이터 분리
    x_columns = ['시가', '고가', '저가', '종가', '거래량'] + training_data.columns.tolist()
    y_columns = ['종가']
    x_data = data[x_columns].values
    y_data = data[y_columns].values
    
    # x_data 정규화
    scaler_x = MinMaxScaler() # minmax 0~1 사이로 정규화
    x_data_norm = scaler_x.fit_transform(x_data)

    # LSTM에 사용할 경우 x_data를 3D로 변환 (samples, timesteps, features)
    x_data_lstm = x_data_norm.reshape(x_data_norm.shape[0], 1, x_data_norm.shape[1])
    
    # CNN에 사용할 경우 x_data를 1D로 변환 (samples, height, width, channels)
    x_data_cnn = x_data_norm.reshape(x_data_norm.shape[0], x_data_norm.shape[1], 1)

    # y_data 정규화
    scaler_y = MinMaxScaler()
    y_data_norm = scaler_y.fit_transform(y_data)

    return x_data_lstm, x_data_cnn, y_data_norm, scaler_x, scaler_y

In [37]:
def split_sequences(sequence, n_steps):
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence) - 1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

def data_setup(n_steps, n_seq, sequence):
    X, y = split_sequences(sequence, n_steps)
    n_features = X.shape[2]
    X = X.reshape((len(X), n_steps, n_features))

    new_y = y[:, -1]
    return X, new_y, n_features

# 입력 데이터와 출력 데이터의 길이 설정
n_steps = 10
n_seq = 10000
rel_test_len = 0.1

# 데이터 설정
values = []
X, y, n_features = data_setup(n_steps, n_seq, values)
X = X[:-1]
y = y[1:]

# 훈련 및 테스트 데이터 분할
X_test, y_test = X[:int(len(X) * rel_test_len)], y[:int(len(X) * rel_test_len)]
X_train, y_train = X[int(len(X) * rel_test_len):], y[int(len(X) * rel_test_len):]

print(X.shape)

IndexError: tuple index out of range

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM

model = Sequential()
model.add(LSTM(64, activation=None, input_shape=(10,4), return_sequences = True))
model.add(LSTM(32, activation=None, return_sequences = True))
model.add(Flatten())
model.add(Dense(100, activation=None))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mse', optimizer='adam')

In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPool1D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(10,4)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mse', optimizer='adam')

In [19]:
import os
from tensorflow.keras import callbacks
epochs = 5000
verbosity = 2
dirx = 'XXXXXXX'
os.chdir(dirx)
h5 = 'network.h5'
checkpoint = callbacks.ModelCheckpoint(h5,
                                       monitor='val_loss',
                                       verbose=0,
                                       save_best_only=True,
                                       save_weights_only=True,
                                       mode='auto',
                                       period=1)
callback = [checkpoint]
json = 'network.json'
model_json = model.to_json()
with open(json, "w") as json_file:
    json_file.write(model_json)
history = model.fit(X_train,
                    y_train,
                    epochs=epochs,
                    batch_size=len(X_train) // 4,
                    validation_data = (X_test,y_test),
                    verbose=verbosity,
                    callbacks=callback)

FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다: 'XXXXXXX'

In [ ]:
from keras.models import load_model, model_from_json
def load_keras_model(optimizer):
    dirx = 'XXXXXXX'
    os.chdir(dirx)
    json_file = open('Convnet.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.compile(optimizer=optimizer, loss='mse')
    model.load_weights('Convnet.h5')
    return model
model = load_keras_model('adam')

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
from matplotlib import pyplot as plt
pred_test = model.predict(X_test)
plt.plot(pred_test,'r')
plt.plot(y_test,'g')